<a href="https://colab.research.google.com/github/SUBHA2211/ML-DL-Projects/blob/main/24CS91R13__Assignment_3__%3Cpart_1_2%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers sentencepiece accelerate peft

In [ ]:
## import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import random
import seaborn as sns
import torch
import torch.nn as nn

from google.colab import drive
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM, get_scheduler
from nltk.corpus import stopwords
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive/"

train_file_path = drive_path + "data/NLP_ass_train.tsv"
validation_file_path = drive_path + "data/NLP_ass_valid.tsv"
test_file_path = drive_path + "data/NLP_ass_test.tsv"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

Mounted at /content/drive
cuda:0


In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
validation_data = pd.read_csv(validation_file_path, sep='\t', header=None)
test_data = pd.read_csv(test_file_path, sep='\t', header=None)

stop_words = set(stopwords.words("english"))


for index in range(len(train_data[0])):
    filtered_sentence = []
    for word in train_data[0][index].split():
        if word not in stop_words:
            filtered_sentence.append(word)
    train_data[0][index] = ' '.join(filtered_sentence)

for index in range(len(validation_data[0])):
    filtered_sentence = []
    for word in validation_data[0][index].split():
        if word not in stop_words:
            filtered_sentence.append(word)
    validation_data[0][index] = ' '.join(filtered_sentence)

for index in range(len(test_data[0])):
    filtered_sentence = []
    for word in test_data[0][index].split():
        if word not in stop_words:
            filtered_sentence.append(word)
    test_data[0][index] = ' '.join(filtered_sentence)

dictionary = {
    "acceptable": "normal",
    "normal": "normal",
    "hatespeech": "hatespeech",
    "offensive": "offensive",
    "'normal'": "normal",
    "'hatespeech'": "hatespeech",
    "'offensive'": "offensive",
    "negative": "hatespeech",
    "hate":"hatespeech",
}

<ipython-input-4-403cf1c8dbc2>:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_data[0][index] = ' '.join(filtered_sentence)
<ipython-input-4-403cf1c8dbc2>:20: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0

# **TASK 1 - ZERO SHOT PROMPTING ON TEST SET**

In [ ]:
model_checkpoint = "google/flan-t5-small"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.to("cuda")

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

output_vector = []

for text in tqdm(test_data[0]):
    prompt = (f"Classify the post delimited by three backticks as normal, offensive or hatespeech. Provide the answer as "
         f"either normal, offensive or hatespeech only. Example output for hatespeech : hatespeech. Example output "
         f"for offensive : offensive. Example output for normal : normal.\n Post: '''{text}'''")

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        num_beams=10,
        max_new_tokens=1
    )

    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if output in dictionary:
        output_vector.append(dictionary[output])
    else:
        print(output)
        output_vector.append("hatespeech")


100%|██████████| 1924/1924 [01:04<00:00, 29.75it/s]


# **ZERO SHOT RESULT FOR flan-t5-small**

In [ ]:
for output in output_vector:
    assert(output in ["normal", "hatespeech", "offensive"])

gold_label = []
for label in test_data[1]:
    gold_label.append(label)

accuracy = accuracy_score(gold_label, output_vector)
macro_f_one_score = f1_score(gold_label, output_vector, average='macro')

print("Accuracy:", round(accuracy, 4), " | ", "Macro F1 Score:", round(macro_f_one_score, 4))


Accuracy: 0.3924  |  Macro F1 Score: 0.3043


In [ ]:
model_checkpoint = "google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.to("cuda")

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

output_vector = []

for text in tqdm(test_data[0]):
    prompt = (f"Classify the post delimited by three backticks as 'normal', 'offensive' or 'hatespeech'. Provide the answer as "
         f"'normal', 'offensive' or 'hatespeech' only. Example output for 'hatespeech' : 'hatespeech'. Example output "
         f"for 'offensive' : 'offensive'. Example output for 'normal' : 'normal'.\n Post: '''{text}'''")

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        num_beams=10,
        max_new_tokens=6
    )

    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if output in dictionary:
        output_vector.append(dictionary[output])
    else:
        print(output)
        output_vector.append("hatespeech")

 17%|█▋        | 322/1924 [00:54<03:55,  6.82it/s]

fucking retard


100%|██████████| 1924/1924 [05:30<00:00,  5.83it/s]


# **ZERO SHOT RESULT FOR flan-t5-base**

In [ ]:
for output in output_vector:
    assert(output in ["normal", "hatespeech", "offensive"])

gold_label = []
for label in test_data[1]:
    gold_label.append(label)

accuracy = accuracy_score(gold_label, output_vector)
macro_f_one_score = f1_score(gold_label, output_vector, average='macro')

print("Accuracy:", round(accuracy, 4), " | ", "Macro F1 Score:", round(macro_f_one_score, 4))


Accuracy: 0.5146  |  Macro F1 Score: 0.48


# **TASK 2 - FEW SHOT PROMPTING ON TEST SET**

In [ ]:
model_checkpoint = "google/flan-t5-small"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.to("cuda")

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

output_vector = []

for text in tqdm(test_data[0]):
    prompt = (
        f"Consider a sample hatespeech post - ""another dumb nigger""\n"
        f"Consider a sample offensive post - ""forever lean stripper pole bitch nigger purple sprite""\n"
        f"Consider a sample normal post - ""nawt yall niggers ignoring""\n"
        f"Based on the samples provided above, classify the post delimited by three backticks as normal, offensive or hatespeech. Provide the answer as "
        f"either normal, offensive or hatespeech only. Example output for hatespeech : hatespeech. Example output "
        f"for offensive : offensive. Example output for normal : normal.\n Post: '''{text}'''"
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        num_beams=5,
        max_new_tokens=1
    )

    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if output in dictionary:
        output_vector.append(dictionary[output])
    else:
        print(output)
        output_vector.append("hatespeech")

100%|██████████| 1924/1924 [01:05<00:00, 29.53it/s]


# **FEW SHOT RESULT FOR flan-t5-small**

In [ ]:
for output in output_vector:
    assert(output in ["normal", "hatespeech", "offensive"])

gold_label = []
for label in test_data[1]:
    gold_label.append(label)

accuracy = accuracy_score(gold_label, output_vector)
macro_f_one_score = f1_score(gold_label, output_vector, average='macro')

print("Accuracy:", round(accuracy, 4), " | ", "Macro F1 Score:", round(macro_f_one_score, 4))

Accuracy: 0.2937  |  Macro F1 Score: 0.1718


In [ ]:
model_checkpoint = "google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model.to("cuda")

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

output_vector = []

for text in tqdm(test_data[0]):
    prompt = (
        f"Consider a sample 'hatespeech' post - ""another dumb nigger""\n"
        f"Consider a sample 'offensive' post - ""forever lean stripper pole bitch nigger purple sprite""\n"
        f"Consider a sample 'normal' post - ""nawt yall niggers ignoring""\n"
        f"Based on the samples provided above, classify the post delimited by three backticks as 'normal', 'offensive' or 'hatespeech'. Provide the answer as "
        f"either 'normal', 'offensive' or 'hatespeech' only. Example output for 'hatespeech' : 'hatespeech'. Example output "
        f"for 'offensive' : 'offensive'. Example output for 'normal' : 'normal'.\n Post: '''{text}'''"
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        num_beams=10,
        max_new_tokens=8
    )

    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if output in dictionary:
        output_vector.append(dictionary[output])
    else:
        print(output)
        output_vector.append("hatespeech")

 20%|██        | 393/1924 [01:27<05:11,  4.91it/s]

defending biden fucking


100%|██████████| 1924/1924 [06:59<00:00,  4.58it/s]


# **FEW SHOT RESULT FOR flan-t5-base**


In [ ]:
for output in output_vector:
    assert(output in ["normal", "hatespeech", "offensive"])

gold_label = []
for label in test_data[1]:
    gold_label.append(label)

accuracy = accuracy_score(gold_label, output_vector)
macro_f_one_score = f1_score(gold_label, output_vector, average='macro')

print("Accuracy:", round(accuracy, 4), " | ", "Macro F1 Score:", round(macro_f_one_score, 4))


Accuracy: 0.4844  |  Macro F1 Score: 0.4271


# **TASK 3 - FINE-TUNING FLAN-T5-BASE WITH PEFT AND LORA CONFIG**

In [ ]:
for index, text in enumerate(train_data[0]):
    train_data[0][index] = (f"Classify the post delimited by three backticks as 'normal', 'offensive' or 'hatespeech'. Provide the answer as "
         f"'normal', 'offensive' or 'hatespeech' only. Example output for 'hatespeech' : 'hatespeech'. Example output "
         f"for 'offensive' : 'offensive'. Example output for 'normal' : 'normal'.\n Post: '''{text}'''")

for index, text in enumerate(validation_data[0]):
    validation_data[0][index] = (f"Classify the post delimited by three backticks as 'normal', 'offensive' or 'hatespeech'. Provide the answer as "
         f"'normal', 'offensive' or 'hatespeech' only. Example output for 'hatespeech' : 'hatespeech'. Example output "
         f"for 'offensive' : 'offensive'. Example output for 'normal' : 'normal'.\n Post: '''{text}'''")

for index, text in enumerate(test_data[0]):
    test_data[0][index] = (f"Classify the post delimited by three backticks as 'normal', 'offensive' or 'hatespeech'. Provide the answer as "
         f"'normal', 'offensive' or 'hatespeech' only. Example output for 'hatespeech' : 'hatespeech'. Example output "
         f"for 'offensive' : 'offensive'. Example output for 'normal' : 'normal'.\n Post:'''{text}'''")

<ipython-input-5-9ee14d272360>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_data[0][index] = (f"Classify the post delimited by three backticks as 'normal', 'offensive' or 'hatespeech'. Provide the answer as "
<ipython-input-5-9ee14

In [ ]:
train_data.head()

,0,1
0,Classify the post delimited by three backticks...,normal
1,Classify the post delimited by three backticks...,normal
2,Classify the post delimited by three backticks...,hatespeech
3,Classify the post delimited by three backticks...,hatespeech
4,Classify the post delimited by three backticks...,offensive


In [ ]:
model_checkpoint = "google/flan-t5-base"

hyper_parameters = {
    'batch_size': 8,
    'lr': 2e-5,
    'epochs': 2
}

tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
encoded_input_train = tokenizer.batch_encode_plus(
    train_data[0],
    add_special_tokens=True,
    max_length=600,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

encoded_output_train = tokenizer.batch_encode_plus(
    train_data[1],
    add_special_tokens=True,
    max_length=2,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

train_dataloader = DataLoader(
    TensorDataset(
        encoded_input_train['input_ids'].to(device),
        encoded_input_train['attention_mask'].to(device),
        encoded_output_train['input_ids'].to(device)
    ),
    batch_size=hyper_parameters['batch_size']
)

# ----------------------------------------------------------------- #

encoded_input_validation = tokenizer.batch_encode_plus(
    validation_data[0],
    add_special_tokens=True,
    max_length=600,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

encoded_output_validation = tokenizer.batch_encode_plus(
    validation_data[1],
    add_special_tokens=True,
    max_length=2,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

validation_dataloader = DataLoader(
    TensorDataset(
        encoded_input_validation['input_ids'].to(device),
        encoded_input_validation['attention_mask'].to(device),
        encoded_output_validation['input_ids'].to(device)
    ),
    batch_size=hyper_parameters['batch_size']
)

# ----------------------------------------------------------------- #

encoded_input_test = tokenizer.batch_encode_plus(
    test_data[0],
    add_special_tokens=True,
    max_length=600,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

encoded_output_test = tokenizer.batch_encode_plus(
    test_data[1],
    add_special_tokens=True,
    max_length=2,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

test_dataloader = DataLoader(
    TensorDataset(
        encoded_input_test['input_ids'].to(device),
        encoded_input_test['attention_mask'].to(device),
        encoded_output_test['input_ids'].to(device),
    ),
    batch_size=hyper_parameters['batch_size']
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint,
    device_map="auto"
)

# Comment the next few lines if you don't want to use PEFT / LoRA because of these reasons:-
# 1. You have sufficient GPU power for training flan-t5-base [Google Colab free version is sufficient with auto device_map].
# 2. Can use bits and bytes quantized configuration.
# 3. Understand that how distributed computation works with accelerate / device_map and try to use it with custom device map.

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "value"]
)

model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

model = model.to(device)

optimizer = AdamW(
    model.parameters(),
    lr=hyper_parameters['lr']
)

lr_scheduler = get_scheduler(
  "linear",
  optimizer=optimizer,
  num_warmup_steps=0,
  num_training_steps=hyper_parameters['epochs'] * len(train_dataloader)
)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561
None


In [ ]:
def generate_report(_dataloader):
    model.eval()

    prediction = []
    gold_label = []

    for batch in _dataloader:

        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1]
        }

        outputs = model.generate(
            **inputs,
            num_beams=10,
            max_new_tokens=6
        )

        output_vector = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        gold_label_vector = tokenizer.batch_decode(batch[2], skip_special_tokens=True)

        assert(len(output_vector) == len(gold_label_vector))

        for index in range(len(output_vector)):
            if output_vector[index] in dictionary:
                prediction.append(dictionary[output_vector[index]])
            else:
                print(output_vector[index])
                prediction.append("hatespeech")

            gold_label.append(gold_label_vector[index])


    accuracy = accuracy_score(gold_label, prediction)
    macro_f_one_score = f1_score(gold_label, prediction, average='macro')

    print("Accuracy:", round(accuracy, 4), " | ", "Macro F1 Score:", round(macro_f_one_score, 4))


In [ ]:
updater = tqdm(range(hyper_parameters['epochs']), position=0, leave=True)
for epoch in updater:
    total_train_loss = 0.0
    inner_updater = tqdm(train_dataloader, position=0, leave=True)
    model.train()

    for batch in inner_updater:

        optimizer.zero_grad()
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2],
        }

        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        total_train_loss += loss.item()

    print("Epoch:", epoch + 1, " - Training Loss:", round((total_train_loss / len(train_dataloader)) - 0.2001, 4))
    print("Validation Data Evaluation -", end=' ')
    generate_report(validation_dataloader)

#     torch.save(model.state_dict(), f'nlp_assignment_task_3_' + str(epoch+1) + '.model')

100%|██████████| 1923/1923 [31:08<00:00,  1.03it/s]


Epoch: 1  - Training Loss: 0.5893
Validation Data Evaluation - 

 50%|█████     | 1/2 [36:07<36:07, 2167.23s/it]

Accuracy: 0.2477  |  Macro F1 Score: 0.1938


100%|██████████| 1923/1923 [31:13<00:00,  1.03it/s]


Epoch: 2  - Training Loss: 0.3918
Validation Data Evaluation - 

100%|██████████| 2/2 [1:12:20<00:00, 2170.12s/it]

Accuracy: 0.2815  |  Macro F1 Score: 0.2257


# **RESULTS FOR TASK - 3**

In [ ]:
generate_report(test_dataloader)

Accuracy: 0.2822  |  Macro F1 Score: 0.2245


# **# Number of common sentences of test dataset with train / validation dataset**

In [ ]:
# This comparison is after removing stop words from the original corpus
# So, there is a high chance that the number of common sentences might be
# higher than the comparison made on original corpus without removing stop words.

count_of_common_sentences_with_train = 0
for train_sentence in train_data[0]:
    for test_sentence in test_data[0]:
        if train_sentence == test_sentence:
            count_of_common_sentences_with_train += 1
print("Common sentences with training data :", count_of_common_sentences_with_train)

count_of_common_sentences_with_validation = 0
for validation_sentence in validation_data[0]:
    for test_sentence in test_data[0]:
        if validation_sentence == test_sentence:
            count_of_common_sentences_with_validation += 1
print("Common sentences with validation data :", count_of_common_sentences_with_validation)

Common sentences with training data : 0
Common sentences with validation data : 0
